# Hotel Review Generator

This notebook will train an SLM to function as a hotel review generator based on structured inputs.

## Dev Recommendation

It's highly recommended that you run this notebook inside a virtual environment. This works best if you have Anaconda, as you can then specify the Python version:

    # Create and activate environment
    conda create -n hotel-reviews python=3.11 -y
    conda activate hotel-reviews

    # Install PyTorch via Conda for handling of CUDA
    conda install pytorch pytorch-cuda=12.1 -c pytorch -c nvidia -y

    # Install and register the Jupyter Notebook kernel inside your env
    pip install ipykernel
    python -m ipykernel install --user --name hotel-reviews --display-name "Hotel Reviews (3.11)"

You should then be able to select the newly installed kernel as your Jupyter kernel.

In [ ]:
# Install packages
!pip install transformers datasets peft trl bitsandbytes accelerate tqdm
#!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --force-reinstall

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig


In [ ]:
# Test environment

import torch
import sys

print(f"Python: {sys.version}")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")

# Test bitsandbytes specifically — this is the one that breaks most often
import bitsandbytes as bnb
print(f"bitsandbytes: {bnb.__version__}")

In [ ]:
# Load the data
import json
with open("preprocessed_reviews.json","r") as f:
    dataset = [json.loads(s) for s in f.readlines()]

print(f"Total dataset size: {len(dataset)} records.")

In [ ]:
# Create a train/test dataset
import random

TRAIN_SIZE = 50000
VAL_SIZE = 2000

SAMPLE_TOTAL = TRAIN_SIZE + VAL_SIZE
dataset_sample = random.sample(dataset, SAMPLE_TOTAL)

dataset_train = dataset_sample[:TRAIN_SIZE]
dataset_val = dataset_sample[TRAIN_SIZE:]

print(f"Train: {len(dataset_train)} records; Val: {len(dataset_val)} records.")

In this initial version, we will focus only on a few features:

- Rating
- Graded since-stay time (0 through 3)

We will expand with amenities and other functionality after we test with this initial setup.

In [ ]:
# Prepare the training dataset by sampling the available data

import tqdm

def create_sample(data_record):

    this_rating = round(data_record['score'])
    stay_latency = data_record['review_elapsed']
    user_request = {
        "rating": this_rating,
        "days_since_stay": stay_latency
    }
    ai_response = f"Positive Review: {data_record['positive']}\n\nNegative Review: {data_record['negative']}"

    final_record = {
        "conversations": [
            {"role": "user", "content": json.dumps(user_request)},
            {"role": "assistant", "content": ai_response}
        ]
    }

    return json.dumps(final_record)

dataset_train_file = [create_sample(x) for x in tqdm.tqdm(dataset_train,desc="Prepare train dataset")]
dataset_val_file = [create_sample(x) for x in tqdm.tqdm(dataset_val,desc="Prepare validation dataset")]

open("train.jsonl","w").write("\n".join(dataset_train_file))
open("val.jsonl","w").write("\n".join(dataset_val_file))

print("Datasets prepared.")

In [ ]:
# Show samples from the dataset
from IPython.display import JSON

JSON(dataset_train_file[0:5])


In [ ]:
# 1. Load base model in 4-bit
try:
    del model
    del trainer
    import gc
    gc.collect()
    torch.cuda.empty_cache()
except:
    pass

import os

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Model dtype: {model.dtype}")
# Also check a specific parameter
for name, param in model.named_parameters():
    print(f"{name} is {param.dtype}")
    break

In [ ]:
dataset = load_dataset("json", data_files={
    "train": "train.jsonl",
    "eval": "val.jsonl",
})

In [ ]:
# 3. Configure LoRA
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# 4. Configure training
training_config = SFTConfig(
    output_dir="./hotel-review-lora",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    warmup_steps=100,
    fp16=False,
    bf16=False,
    logging_steps=50,
    eval_strategy="steps",
    eval_steps=500,
    save_strategy="epoch",
    #max_seq_length=768,
    dataset_text_field="text"
)

In [ ]:
# Format the dataset as required
def format_chat(example):
    messages = example["conversations"]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False,
    )
    return {"text": text}

dataset = dataset.map(format_chat, remove_columns=["conversations"])

print("Dataset preparation completed.")

# Verify it looks right
print(dataset["train"][0])


# Train the model

In [ ]:
# 5. Create trainer and run
trainer = SFTTrainer(
    model=model,
    args=training_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["eval"],
    peft_config=lora_config,
    processing_class=tokenizer,
)

trainer.train()

In [ ]:
trainer.save_model("./hotel-review-lora")
tokenizer.save_pretrained("./hotel-review-lora")


In [ ]:
# Training is complete!
# Clear the environment

del trainer
del model
torch.cuda.empty_cache()

# Test the model

In [ ]:
# Load the new model

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Load base model in 4-bit (same as training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=bnb_config,
    device_map="auto",  # Automatically places layers on available GPUs
)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")

# Apply your LoRA adapter on top
model = PeftModel.from_pretrained(base_model, "./hotel-review-lora")
model.eval()


In [ ]:
def generate(rating: int, days_since_stay: int):
    # Generate
    
    prompt = {"rating": rating, "days_since_stay": days_since_stay}
    prompt = json.dumps(prompt)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            repetition_penalty=1.1,
        )
    
    review = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return review

for i in tqdm.trange(10):
    print(generate(i+1, i))


In [1]:
# Merge the model into a single completed model prior to GGUF conversion

from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load base model at full precision for merging
base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    torch_dtype=torch.float16,
    device_map="cpu",  # Merge on CPU to avoid VRAM issues
)

# Apply LoRA adapter
model = PeftModel.from_pretrained(base_model, "./hotel-review-lora")

# Merge weights
merged_model = model.merge_and_unload()

# Save
merged_model.save_pretrained("./Mistral-HotelReviews-7b")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3")
tokenizer.save_pretrained("./Mistral-HotelReviews-7b")

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Writing model shards: 100%|██████████| 1/1 [01:42<00:00, 102.80s/it]


('./Mistral-HotelReviews-7b/tokenizer_config.json',
 './Mistral-HotelReviews-7b/chat_template.jinja',
 './Mistral-HotelReviews-7b/tokenizer.json')